In [1]:
import os
from scipy import stats
from process import *
from utils import *

/home/hongn/miniconda3/envs/xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pre-defined

In [2]:
TRIALS = {
    "all": TRIAL_LIST,
    "texture": TRIAL_TEXTURE,
    "complexity": TRIAL_COMPLEXITY,
    "orientation": TRIAL_ORIENTATION,
    "brightness": TRIAL_BRIGHTNESS,
    "contrast": TRIAL_CONTRAST,
    "color": TRIAL_COLOR,
    "depth": TRIAL_DEPTH,
    "face": TRIAL_FACE,
    "movement": TRIAL_MOVEMENT,
    "face_texture": TRIAL_FACE_TEXTURE,
}

these_trials = TRIALS["face"]
smap = "face"

def statistic_mw(x, y):
    return stats.mannwhitneyu(x, y)[0]


def iqr(x):
    q1, q3 = np.percentile(x, [25, 75])
    return q3 - q1


def extract_features(ids, trial, smap):
    if smap is not None:
        smap = trial.load_saliency_map(smap)
        smap_vs = trial.load_saliency_map("red")

    features = []
    for subject in ids:
        sub = Subject(DATA_ROOT, subject)
        out = sub.extract_fixations(trial_name=this_trial.trial_name)
        analyzer = FixationAnalyzer(DATA_ROOT, out)
        feat = analyzer.average_saliency(smap)
        feat_vs = analyzer.average_saliency(smap_vs)
        # feat = analyzer.number_of_fixations()
        # feat = sub.eye_mov_entropy(trial.trial_name, perplexity=True)
        features.append(feat - feat_vs)
        feats[subject].append(feat - feat_vs)
    return features

In [3]:
class ImageTrial:
    def __init__(self, root, trial_name, smap_dir):
        self.root = root
        self.new_res = True if "new_res" in root else False
        self.path = glob.glob(f"/mnt/c/Users/PCM/Dropbox/cvi-stimuli/*{trial_name}*")[0]
        self.trial_name = trial_name
        self.smap_dir = smap_dir
        self.ids = glob.glob(os.path.join(root, "*.asc"))
        self.ids = [os.path.basename(d)[:-4] for d in self.ids]

    def load_trial_img(self):
        img = cv2.imread(self.path)
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    def load_saliency_map(self, smap_type):
        image_name = self.trial_name.strip(".jpg")
        filename = f"{image_name}_{smap_type}.jpg"
        path = os.path.join(self.smap_dir, image_name, filename)

        if os.path.exists(path):
            smap = plt.imread(path)
        else:
            os.makedirs(os.path.join(self.smap_dir, image_name), exist_ok=True)
            sal = SaliencyMap(smap_type)
            smap = sal.get_smap(self.load_trial_img())
            cv2.imwrite(path, smap)

        if self.new_res:
            smap = np.pad(smap, ((240, 240), (320, 320)), 'constant')
        return smap.T

    def complexity(self):
        img = self.load_trial_img()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        stats = measure.regionprops(img)
        areas = [l.area for l in stats]
        rp_tot = img.shape[0] * img.shape[1]
        return sum(areas > (rp_tot / 25000))

    def read_subjects(self, names, vel=False):
        data, frac = {}, {}
        for subject in names:
            sub = Subject(subject)
            trial_data, frac = sub.extract_data(self.trial_name, vel)
            data[subject] = trial_data
            frac[subject] = 1 - frac
        return data, frac

    def read_fixations(self, names):
        fixations = {}
        for subject in names:
            sub = Subject(subject)
            this = sub.extract_fixations(self.trial_name)
            fixations[subject] = this
        return fixations

    def extract_traces(self, names, smap):
        traces = {}
        for subject in names:
            sub = Subject(subject)
            this = sub.extract_trace(self.trial_name, smap)
            traces[subject] = this
        return traces

## Testing

In [4]:
DATA_ROOT = "/mnt/c/Users/PCM/Dropbox/cvi-asc/"

In [5]:
ids = [i for i in os.listdir(DATA_ROOT) if i.endswith(".asc")]
ctrl_ids = [i.split(".")[0] for i in ids if i.startswith("2")]
cvi_ids = [i.split(".")[0] for i in ids if i.startswith("1")]
print(f'number of control {len(ctrl_ids)} and cvi {len(cvi_ids)}')

number of control 10 and cvi 31


In [6]:
feats = {name.split(".")[0]: [] for name in ids}
trial = these_trials[2]
trial

'Freeviewingstillimage_5'

In [7]:
this_trial = ImageTrial(DATA_ROOT, trial, "smaps")
# features_ctrl = extract_features(ctrl_ids, this_trial, smap)
# features_cvi = extract_features(cvi_ids, this_trial, smap)

In [8]:
subject = cvi_ids[0]
sub = Subject(DATA_ROOT, subject)
out = sub.extract_fixations(trial_name='Freeviewingstillimage_2')
out_saccade = sub.extract_saccades(trial_name=this_trial.trial_name)
analyzer = FixationAnalyzer(DATA_ROOT, out)

In [11]:
out_saccade

[{'data': array([[753, 368],
         [756, 367],
         [762, 365],
         [769, 364],
         [777, 363],
         [785, 362],
         [793, 360],
         [800, 359],
         [808, 358],
         [814, 357],
         [820, 357],
         [824, 356]]),
  'fraction': 0.08333333333333333,
  'latency': 1388,
  'duration': 22}]

In [11]:
trainctmc = []
trainctmc.append('init')

In [9]:
sub.get_trial()

['2529046\t  320.2\t  362.1\t  323.0\t   .\t   .\t    0.0\t    0.0\t..... \t 4700.0\t 7334.0\t  654.6 .................\n',
 'SSACC R  2529048\n',
 '2529048\t  320.9\t  362.2\t  323.0\t   .\t   .\t    0.0\t    0.0\t..... \t 4698.0\t 7335.0\t  654.5 .................\n',
 '2529050\t  321.4\t  362.3\t  324.0\t   .\t   .\t    0.0\t    0.0\t..... \t 4696.0\t 7336.0\t  654.5 .................\n',
 '2529052\t  322.0\t  362.4\t  324.0\t   .\t   .\t    0.0\t    0.0\t..... \t 4695.0\t 7337.0\t  654.4 .................\n',
 '2529054\t  322.6\t  362.2\t  325.0\t   .\t   .\t    0.0\t    0.0\t..... \t 4693.0\t 7338.0\t  654.5 .................\n',
 'SFIX L   2529056\n',
 '2529056\t  322.9\t  361.1\t  325.0\t   .\t   .\t    0.0\t    0.0\t..... \t 4692.0\t 7339.0\t  654.5 .................\n',
 '2529058\t  322.7\t  358.3\t  326.0\t   .\t   .\t    0.0\t    0.0\t..... \t 4691.0\t 7340.0\t  654.5 .................\n',
 '2529060\t  322.3\t  354.2\t  327.0\t   .\t   .\t    0.0\t    0.0\t..... \t 4689.0\t 

In [10]:
sub.get_CTMC()

['init',
 44,
 'SBLINK',
 182,
 'ESACC',
 144,
 'EFIX',
 44,
 'ESACC',
 152,
 'EFIX',
 26,
 'SBLINK',
 106,
 'ESACC',
 516,
 'EFIX',
 26,
 'ESACC',
 262,
 'EFIX',
 24,
 'ESACC',
 158,
 'EFIX',
 30,
 'SBLINK']

In [29]:
dataa = sub.extract_data(trial_name='Freeviewingstillimage_2')

In [24]:
sub

In [28]:
out

[{'data': array([[632, 364],
         [633, 363],
         [634, 362],
         [634, 362],
         [634, 361],
         [634, 360],
         [634, 360],
         [634, 360],
         [634, 361],
         [634, 361],
         [634, 361],
         [635, 361],
         [635, 361],
         [635, 361],
         [635, 361],
         [635, 361],
         [635, 361],
         [635, 360],
         [636, 358],
         [636, 357],
         [636, 357],
         [636, 356],
         [637, 357],
         [638, 358],
         [638, 360],
         [639, 363],
         [639, 365],
         [640, 367],
         [641, 369],
         [642, 369],
         [642, 369],
         [643, 370],
         [643, 370],
         [644, 371],
         [645, 372],
         [645, 374],
         [646, 376],
         [646, 377],
         [647, 378],
         [648, 380],
         [648, 381],
         [648, 382],
         [648, 384],
         [648, 386],
         [648, 387],
         [648, 389],
         [648, 390],
     

In [11]:
out

[{'data': array([[632, 364],
         [633, 363],
         [634, 362],
         [634, 362],
         [634, 361],
         [634, 360],
         [634, 360],
         [634, 360],
         [634, 361],
         [634, 361],
         [634, 361],
         [635, 361],
         [635, 361],
         [635, 361],
         [635, 361],
         [635, 361],
         [635, 361],
         [635, 360],
         [636, 358],
         [636, 357],
         [636, 357],
         [636, 356],
         [637, 357],
         [638, 358],
         [638, 360],
         [639, 363],
         [639, 365],
         [640, 367],
         [641, 369],
         [642, 369],
         [642, 369],
         [643, 370],
         [643, 370],
         [644, 371],
         [645, 372],
         [645, 374],
         [646, 376],
         [646, 377],
         [647, 378],
         [648, 380],
         [648, 381],
         [648, 382],
         [648, 384],
         [648, 386],
         [648, 387],
         [648, 389],
         [648, 390],
     

In [22]:
def extract_features(ids, trial, smap):
    if smap is not None:
        smap = trial.load_saliency_map(smap)
        smap_vs = trial.load_saliency_map("red")

    features = []
    for subject in ids:
        sub = Subject(DATA_ROOT, subject)
        out = sub.extract_fixations(trial_name=this_trial.trial_name)
        analyzer = FixationAnalyzer(DATA_ROOT, out)
        feat = analyzer.average_saliency(smap)
        feat_vs = analyzer.average_saliency(smap_vs)
        # feat = analyzer.number_of_fixations()
        # feat = sub.eye_mov_entropy(trial.trial_name, perplexity=True)
        features.append(feat - feat_vs)
        feats[subject].append(feat - feat_vs)
    return features